___
## PlotlyCandles
### Use Plot.ly python Vertical Subplots and Shapes to create candlestick charts that humans can understand and modify.

Plot.ly has a nice Candlestick trace type (as opposed to Scatter, Bar, etc).  Unfortunately, their candlestick code was too complicated to modify for my simple mind.  Therefore, I built a PlotlyCandles class that I found:
1. more flexible, easier to modify and 
2. more illustritive of the uses of Subplots and Graph Object Shapes than the overly simply examples in the https://www.plot.ly/python documentation.
___
The PlotlyCandles combines the idea of Subplots (arranged vertically) and the concept of Shapes, which allow you to create interesting graphical objects arount the points of a scatter plot.  The central trick to pull this off is to NOT use the x values that you want to actually show on the xaxis, but use simple index values from 1 to the length of your input DataFrame (len(df)).  You then use the ```tickvals``` and ```ticktext``` attributes of the ```plotly.graph_objs.Layout``` class to force Plotly to display actual date values instead of indices like ```[1,2,3, ..., len(df)]```

```
        layout1 = go.Layout(
            showlegend=False,
            title = title,
            margin = dict(t=100),
            xaxis = go.layout.XAxis(
                tickmode = 'array',
                # ---------------- USE THESE 2 ATTRIBUTES BELOW TO DISPLAY XAXIS VALUES ---------------
                tickvals = indices,
                ticktext = tdvals,
                # -------------------------------------------------------------------------------------
                tickangle=90,
                showgrid = True,
                showticklabels=True,
                anchor='y2', 
            ),       
            yaxis1 = go.layout.YAxis(
                range =  [min(df.low.values), max(df.high.values)],
                domain=[.22,1]
            ),
            yaxis2 = go.layout.YAxis(
                range =  [0, max(df.volume.values)],
                domain=[0,.2]
            ),
            shapes = shapes
        )
```




In [2]:
import numpy as np
import pandas as pd
import pandas_datareader.data as pdr
import datetime
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)


In [3]:
class PlotlyCandles():
    BAR_WIDTH=.5
    def __init__(self,df,title='candle plot',number_of_ticks_display=20):
        self.df = df.copy()
        #  and make sure the first index is 1 NOT 0!!!!!!
        self.df.index = np.array(list(range(len(df))))+1
        
        self.title = title
        self.number_of_ticks_display = number_of_ticks_display
        
    def get_candle_shapes(self):
        df = self.df.copy()
        xvals = df.index.values #chg    
        lows = df.low.values
        highs = df.high.values
        closes = df.close.values
        opens = df.open.values
        df['is_red'] = df.open>=df.close
        is_reds = df.is_red.values
        lines_below_box = [{
                    'type': 'line',
                    'x0': xvals[i],
                    'y0': lows[i],
                    'x1': xvals[i],
                    'y1': closes[i] if is_reds[i] else opens[i],
                    'line': {
                        'color': 'rgb(55, 128, 191)',
                        'width': 1.5,
                    }
                } for i in range(len(xvals))
        ]

        lines_above_box = [{
                    'type': 'line',
                    'x0': xvals[i],
                    'y0': opens[i] if is_reds[i] else closes[i],
                    'x1': xvals[i],
                    'y1': highs[i],
                    'line': {
                        'color': 'rgb(55, 128, 191)',
                        'width': 1.5,
                    }
                }for i in range(len(xvals))
        ]


        boxes = [{
                    'type': 'rect',
                    'xref': 'x',
                    'yref': 'y',
                    'x0': xvals[i]- PlotlyCandles.BAR_WIDTH/2,
                    'y0': closes[i] if is_reds[i] else opens[i],
                    'x1': xvals[i]+ PlotlyCandles.BAR_WIDTH/2,
                    'y1': opens[i] if is_reds[i] else closes[i],
                    'line': {
                        'color': 'rgb(55, 128, 191)',
                        'width': 1,
                    },
                    'fillcolor': 'rgba(255, 0, 0, 0.6)' if is_reds[i] else 'rgba(0, 204, 0, 0.6)',
                } for i in range(len(xvals))
        ]
        shapes = lines_below_box + boxes + lines_above_box
        return shapes

    
    def get_figure(self):
        '''
        Use Plotly to create a financial candlestick chart.
        The DataFrame df_in must have columns called:
         'date','open','high','low','close'
        '''
        # Step 0: get important constructor values (so you don't type 'self.'' too many times)
        df_in = self.df.copy()
        title=self.title
        number_of_ticks_display=self.number_of_ticks_display
        
        # Step 1: only get the relevant columns and sort by date
        cols_to_keep = ['date','open','high','low','close','volume']
        df = df_in[cols_to_keep].sort_values('date')
        # Step 2: create a data frame for "green body" days and "red body" days
        # Step 3: create the candle shapes that surround the scatter plot in trace1
        shapes = self.get_candle_shapes()

        # Step 4: create an array of x values that you want to show on the xaxis
        spaces = len(df)//number_of_ticks_display
        indices = list(df.index.values[::spaces]) + [max(df.index.values)]
        tdvals = df.loc[indices].date.values

        # Step 5: create a layout
        layout1 = go.Layout(
            showlegend=False,
            title = title,
            margin = dict(t=100),
            xaxis = go.layout.XAxis(
                tickmode = 'array',
                tickvals = indices,
                ticktext = tdvals,
                tickangle=90,
                showgrid = True,
                showticklabels=True,
                anchor='y2', 
            ),       
            yaxis1 = go.layout.YAxis(
                range =  [min(df.low.values), max(df.high.values)],
                domain=[.22,1]
            ),
            yaxis2 = go.layout.YAxis(
                range =  [0, max(df.volume.values)],
                domain=[0,.2]
            ),
            shapes = shapes
        )

        # Step 6: create a scatter object, and put it into an array
        def __hover_text(r):
            d = r.date
            o = r.open
            h = r.high
            l = r.low
            c = r.close
            v = r.volume
            t = f'date: {d}<br>open: {o}<br>high: {h}<br>low: {l}<br>close: {c}<br>volume: {v}' 
            return t
        df['hover_text'] = df.apply(__hover_text,axis=1)
        hover_text = df.hover_text.values

        # Step 7: create scatter (close values) trace.  The candle shapes will surround the scatter trace
        trace1 = go.Scatter(
            x=df.index.values,
            y=df.close.values,
            mode = 'markers',
            text = hover_text,
            hoverinfo = 'text',
            xaxis='x',
            yaxis='y1'
        )

        # Step 8: create the bar trace (volume values)
        trace2 = go.Bar(
            x=df.index.values,
            y=df.volume.values,
            width = PlotlyCandles.BAR_WIDTH,
            xaxis='x',
            yaxis='y2'
        )

        # Step 9: create the final figure and pass it back to the caller
        fig1 = {'data':[trace1,trace2],'layout':layout1}
        return fig1
    
    def plot(self):
        fig = self.get_figure()
        iplot(fig)
        return fig

In [4]:
num_vals = 50
closes = np.random.randint(5,20,size=num_vals)
opens = closes - np.random.randint(1,4,size=num_vals)*np.random.choice([-1,1],size=num_vals)
highs = [max(closes[i],opens[i]) + np.random.randint(1,4) for i in range(num_vals)]
lows = [max(0,min(closes[i],opens[i]) - np.random.randint(1,4)) for i in range(num_vals)]
volumes = np.random.randint(1,10000,size=num_vals)
dt_end = datetime.datetime.now()
dt_beg = dt_end - datetime.timedelta(num_vals*2)
dates = pd.date_range(dt_beg,dt_end,freq='B')
dates = dates[-num_vals:]
dates = dates.astype(str).str.slice(0,10)

# df_all_days = pd.DataFrame({'date':np.array(range(len(closes)))+1000,'open':opens,'high':highs,'low':lows,'close':closes})
df_all_days = pd.DataFrame({'date':dates,'open':opens,'high':highs,'low':lows,'close':closes,'volume':volumes})

fig1 = PlotlyCandles(df_all_days,number_of_ticks_display=10,title='random values').plot()


In [5]:
num_vals = 100
dt_end = datetime.datetime.now()
dt_beg = dt_end - datetime.timedelta(num_vals)
df = pdr.DataReader('AAPL', 'yahoo', dt_beg, dt_end)
df['date'] = df.index
df.index = list(range(len(df)))
df = df.rename(columns={c:c.lower() for c in df.columns.values})
df = df[['date','open','high','low','close','volume']]
df.date = df.date.astype(str).str.slice(0,10)
t = f'AAPL from {df.iloc[0].date} to {df.iloc[len(df)-1].date}'
fig1 = PlotlyCandles(df,number_of_ticks_display=10,title=t).plot()


In [7]:
stocks = ['MSFT','GOOGL','FB','AMZN']
num_vals = 100
dt_end = datetime.datetime.now()
dt_beg = dt_end - datetime.timedelta(num_vals)

for stock in stocks:
    df = pdr.DataReader(stock, 'yahoo', dt_beg, dt_end)
    df['date'] = df.index
    df.index = list(range(len(df)))
    df = df.rename(columns={c:c.lower() for c in df.columns.values})
    df = df[['date','open','high','low','close','volume']]
    df.date = df.date.astype(str).str.slice(0,10)
    t = f'{stock} from {df.iloc[0].date} to {df.iloc[len(df)-1].date}'
    fig1 = PlotlyCandles(df,number_of_ticks_display=10,title=t).plot()
    